In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import time
from time import localtime, strftime
import bs4
import requests
 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

# Đọc và chuẩn bị dữ liệu

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
#Đọc dữ liệu
import pandas as pd
data = pd.read_csv('final_data.csv')
data

,Hour,Temperature,Forecast,Rain,Rain_rate,Cloud_rate,Pressure,Wind,Gust,Day,Month,Year,Weather
0,0,26.0,28.0,0.0,0.0,0.10,1015.0,8.0,11.0,1,1,2020,0
1,3,25.0,26.0,0.0,0.0,0.19,1014.0,8.0,13.0,1,1,2020,0
2,6,24.0,26.0,0.0,0.0,0.10,1015.0,8.0,11.0,1,1,2020,0
3,9,29.0,30.0,0.0,0.0,0.05,1016.0,7.0,15.0,1,1,2020,0
4,12,34.0,34.0,0.0,0.0,0.08,1015.0,10.0,17.0,1,1,2020,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16523,9,32.0,36.0,0.0,0.0,0.21,1010.0,12.0,14.0,30,4,2023,0
16524,12,37.0,42.0,0.0,0.0,1.00,1007.0,9.0,10.0,30,4,2023,0
16525,15,33.0,37.0,0.4,0.0,0.84,1005.0,12.0,15.0,30,4,2023,1
16526,18,30.0,33.0,0.7,0.0,0.82,1006.0,17.0,26.0,30,4,2023,1


In [4]:
# #Cách 1
# features = data.drop(['Weather'], axis=1)
# target = data.Weather

In [5]:
#Cách 2
features = data.drop(['Weather'], axis=1).iloc[:-1, :]
target = data.Weather.iloc[1:]

In [6]:
# # Cách 3
# features = data.drop(['Day', 'Month', 'Year', 'Weather'], axis=1).iloc[:-1, :]
# target = data.Weather.iloc[1:]

In [7]:
# # Cách 4
# features = data.drop(['Day', 'Year', 'Weather'], axis=1).iloc[:-1, :]
# target = data.Weather.iloc[1:]

# rain_ss = [5,6,7,8,9,10,11]
# norain_ss = [12,1,2,3,4]
# features['Month'] = features['Month'].replace(norain_ss, 0)
# features['Month'] = features['Month'].replace(rain_ss, 1)
# features['Month'].value_counts()

In [8]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16527 entries, 0 to 16526
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Hour         16527 non-null  int64  
 1   Temperature  16527 non-null  float64
 2   Forecast     16527 non-null  float64
 3   Rain         16527 non-null  float64
 4   Rain_rate    16527 non-null  float64
 5   Cloud_rate   16527 non-null  float64
 6   Pressure     16527 non-null  float64
 7   Wind         16527 non-null  float64
 8   Gust         16527 non-null  float64
 9   Day          16527 non-null  int64  
 10  Month        16527 non-null  int64  
 11  Year         16527 non-null  int64  
dtypes: float64(8), int64(4)
memory usage: 1.5 MB


In [9]:
target.info()

<class 'pandas.core.series.Series'>
RangeIndex: 16527 entries, 1 to 16527
Series name: Weather
Non-Null Count  Dtype
--------------  -----
16527 non-null  int64
dtypes: int64(1)
memory usage: 129.2 KB


In [10]:

X_train, X_val, Y_train, Y_val = train_test_split(features,
                                                  target,
                                                  test_size = 0.2,
                                                  stratify = target,
                                                  random_state=2)
 
# lặp lại các dòng dữ liệu có số lượng label thấp hơn để tăng tính cân bằng
ros = RandomOverSampler(sampling_strategy='minority', random_state=22)
X, Y = ros.fit_resample(X_train,Y_train)

In [11]:
# chuẩn hóa toàn bộ dữ liệu
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_val = scaler.transform(X_val)

# Chạy các mô hình máy học

In [12]:
models = [LogisticRegression(), XGBClassifier(), SVC(probability=True)]

for i in range(3):
  st = time.time()
  models[i].fit(X, Y)
  et = time.time()

  print(f'{models[i]} : ')

  train_preds = models[i].predict_proba(X)
  print('Training Accuracy : ', metrics.roc_auc_score(Y, train_preds[:,1]))

  val_preds = models[i].predict_proba(X_val)
  print('Validation Accuracy : ', metrics.roc_auc_score(Y_val, val_preds[:,1]))

  print("Time: ", et-st)

  # cm = metrics.confusion_matrix(Y_val, models[i].predict(X_val), labels=models[i].classes_)
  # disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=models[i].classes_)
  # disp.plot()
  print("==========================================================")



LogisticRegression() : 
Training Accuracy :  0.8283663744859514
Validation Accuracy :  0.8344128868322417
Time:  0.01900482177734375


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...) : 
Training Accuracy :  0.9947025462464109
Validation Accuracy :  0.9515248809603649
Time:  0.3725883960723877
SVC(probability=True) : 
Training Accuracy :  0.8951891264236178
Validation Accuracy :  0.8812635526345204
Time:  42.93954944610596


In [13]:
print(metrics.classification_report(Y_val, models[1].predict(X_val), digits=5))

              precision    recall  f1-score   support

           0    0.94637   0.90488   0.92516      2418
           1    0.76861   0.86036   0.81190       888

    accuracy                        0.89292      3306
   macro avg    0.85749   0.88262   0.86853      3306
weighted avg    0.89862   0.89292   0.89474      3306



# Demo

In [14]:
def get_page_content(url):
   page = requests.get(url,headers={"Accept-Language":"en-US"})
   return bs4.BeautifulSoup(page.text,"html.parser")

In [15]:
real_time = strftime("%Y-%m-%d %H:%M:%S", localtime())
year, month, day, hour = real_time[0:4], real_time[5:7], real_time[8:10], real_time[11:13]
print(year, month, day, hour)

url = f'https://www.worldweatheronline.com/ho-chi-minh-city-weather/vn.aspx?day={day}&tp=1' #link website
soup = get_page_content(url)
weather_now = soup.findAll('tr')[int(hour)+2].get_text()

datatest = [[int(weather_now[0:2]), int(weather_now[5:7]), int(weather_now[10:12]), float(weather_now[15:18]), float(weather_now[20:21])/100, 
             float(weather_now[22:24])/100, int(weather_now[25:29]), int(weather_now[31:33]), int(weather_now[38:40]), int(day), int(month), int(year)]]
datatest

2023 06 20 18


[[18, 31, 36, 0.0, 0.0, 0.33, 1006, 13, 20, 20, 6, 2023]]

In [22]:
datatest = scaler.transform(datatest)
y_test = models[1].predict(datatest)

c:\Users\nife\miniconda3\envs\DS103\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [23]:
if y_test[0]==0:
    print(f'{int(hour)+3}:00 {day}-{month}-{year}: Không có mưa!')
else:
    print(f'{int(hour)+3}:00 {day}-{month}-{year}: Sẽ có mưa!')

21:00 20-06-2023: Không có mưa!
